In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="yB1sjDLxzVT1r0Q4ZMiW")
project = rf.workspace("new-workspace-ienyy").project("personal-protective-equipments-cmu6w")
version = project.version(7)
dataset = version.download("yolov11")


In [3]:
import os

In [4]:
%pwd

'c:\\Users\\Dell\\Desktop\\YOLO-Custom-Object-Detection-Web-App-in-Python\\research'

In [5]:
os.chdir("../")

In [9]:
%pwd

'c:\\Users\\Dell\\Desktop\\YOLO-Custom-Object-Detection-Web-App-in-Python'

In [7]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from src.ppeDetection.constants import *
from src.ppeDetection.utils.common import read_yaml, create_directories

In [12]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH, schema_filepath=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Nếu có tùy chọn Roboflow, thay thế API key trong URL
        if config.roboflow_option:
            source_URL = config.source_URL.replace("${ROBOFLOW_API_KEY}", os.getenv("ROBOFLOW_API_KEY"))
        else:
            source_URL = config.source_URL

        # Tạo thư mục lưu trữ dữ liệu
        create_directories([config.root_dir])

        # Tạo đối tượng DataIngestionConfig
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=source_URL,  # Sử dụng URL đã chỉnh sửa
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [14]:
import os
import urllib.request as request
import zipfile
from src.ppeDetection import logger
from src.ppeDetection.utils.common import get_size

In [19]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted {self.config.local_data_file} to {unzip_path}")
    
    def remove_zip_file(self):
        os.remove(self.config.local_data_file)
        logger.info(f"Removed {self.config.local_data_file}")

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.remove_zip_file()
except Exception as e:
    raise e

[2025-01-08 10:44:14,962: INFO: common: yaml file: config\config.yaml loaded successfully
[2025-01-08 10:44:14,962: INFO: common: yaml file: params.yaml loaded successfully
[2025-01-08 10:44:14,964: INFO: common: yaml file: schema.yaml loaded successfully
[2025-01-08 10:44:14,965: INFO: common: created directory at: artifacts
[2025-01-08 10:44:14,967: INFO: common: created directory at: artifacts/data_ingestion
[2025-01-08 10:46:59,796: INFO: 2441967934: artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: AFiumC6Y3_ZRlGS9aFI2O4rwFS2VK8S39VbGovNEGgXk3QV6mLNO2ZL63sDHfGv4nPyxZk0
Expires: Wed, 08 Jan 2025 03:44:39 GMT
Date: Wed, 08 Jan 2025 03:44:39 GMT
Cache-Control: private, max-age=0
Last-Modified: Fri, 03 Jan 2025 07:00:38 GMT
ETag: "5faa850728c222fa6448e1086be82699"
x-goog-generation: 1735887638700893
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 140351362
Content-Type: application/zip
Content-Dispos